In [1]:
import os
import json
import math
import torch
import random
import osmium
from osmium import osm
from typing import List, Tuple
from torch_geometric.nn import TransformerConv
from torch_geometric.data import Data
from torch_geometric.utils import to_dense_adj
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import torch.nn as nn
import networkx as nx

R = 6371000  


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def construct_graph(osmPath: str):

    def compute_distance(lat1, lon1, lat2, lon2):
        phi1 = math.radians(lat1)
        phi2 = math.radians(lat2)
        dphi = math.radians(lat2 - lat1)
        dlambda = math.radians(lon2 - lon1)
        a = (math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2)
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return R * c

    class MapCreationHandler(osmium.SimpleHandler):
        def __init__(self) -> None:
            super().__init__()
            self.nodes = []
            self.edges = [[], []]
            self.edge_dist = []
            self.node_id_to_idx = {}
            self.idx_to_node_id = {}
            self.id_counter = 0

        def node(self, n: osmium.osm.Node) -> None:
            self.nodes.append([n.location.lat, n.location.lon])
            self.node_id_to_idx[n.id] = self.id_counter
            self.idx_to_node_id[self.id_counter] = n.id
            self.id_counter += 1

        def way(self, w):
            node_refs = [node.ref for node in w.nodes]

            for i in range(len(node_refs) - 1):
                node_start = node_refs[i]
                node_end = node_refs[i + 1]

                node_1_idx = self.node_id_to_idx[node_start]
                node_2_idx = self.node_id_to_idx[node_end]

                self.edges[0].append(node_1_idx)
                self.edges[1].append(node_2_idx)

                node_1 = self.nodes[node_1_idx]
                node_2 = self.nodes[node_2_idx]

                n1_lat, n1_lon = node_1
                n2_lat, n2_lon = node_2

                dist = compute_distance(n1_lat, n1_lon, n2_lat, n2_lon)
                self.edge_dist.append(dist)

    mapCreator = MapCreationHandler()
    mapCreator.apply_file(osmPath, locations=True)

    x = torch.tensor(mapCreator.nodes, dtype=torch.float)
    edge_index = torch.tensor(mapCreator.edges, dtype=torch.long)
    edge_attr = torch.tensor(mapCreator.edge_dist, dtype=torch.float).unsqueeze(1)
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

    return data, mapCreator.node_id_to_idx

In [3]:
class PathDataset(Dataset):

    def __init__(self, route_files: List[str], node_id_to_idx: dict, shuffle_waypoints: bool = True, fixed_length: int = 8):
        self.route_files = route_files
        self.node_id_to_idx = node_id_to_idx
        self.shuffle_waypoints = shuffle_waypoints
        self.fixed_length = fixed_length
        self.data = self._load_data()

    def _load_data(self):
        samples = []
        for f in self.route_files:
            with open(f, 'r') as json_file:
                route_info = json.load(json_file)

            start_id = route_info["start"]
            end_id = route_info["end"]
            waypoint_tags = route_info["waypointTags"]
            waypoint_ids = [tag.split('=')[1] for tag in waypoint_tags]

            try:
                start_idx = self.node_id_to_idx[int(start_id)]
                end_idx = self.node_id_to_idx[int(end_id)]
                waypoints_correct = [self.node_id_to_idx[int(w_id)] for w_id in waypoint_ids]
            except KeyError:
                continue

            samples.append({
                "start_idx": start_idx,
                "waypoints_correct": waypoints_correct,
                "end_idx": end_idx
            })
        return samples
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):
        sample = self.data[idx]
        start_idx = torch.tensor(sample["start_idx"], dtype=torch.long)
        end_idx = torch.tensor(sample["end_idx"], dtype=torch.long)
        waypoints_correct = sample["waypoints_correct"]

        # Pad way points if length is different
        if len(waypoints_correct) < self.fixed_length:
            padding_needed = self.fixed_length - len(waypoints_correct)
            waypoints_correct = waypoints_correct + [end_idx.item()] * padding_needed

        waypoints_correct_tensor = torch.tensor(waypoints_correct, dtype=torch.long)

        if self.shuffle_waypoints and len(waypoints_correct) > 1:
            waypoints_shuffled = waypoints_correct[:]

            # Random change node order
            random.shuffle(waypoints_shuffled)
            waypoints_shuffled_tensor = torch.tensor(waypoints_shuffled, dtype=torch.long)
        else:
            waypoints_shuffled_tensor = waypoints_correct_tensor.clone()

        return start_idx, waypoints_shuffled_tensor, waypoints_correct_tensor, end_idx

In [4]:
def load_path_data(route_dir: str, node_id_to_idx: dict, train_ratio=0.8, val_ratio=0.1, seed=42):

    torch.manual_seed(seed)

    route_files = [os.path.join(route_dir, f) for f in os.listdir(route_dir) if f.endswith('.json')]
    full_dataset = PathDataset(route_files, node_id_to_idx, shuffle_waypoints=False)
    dataset_len = len(full_dataset)
    train_len = int(dataset_len * train_ratio)
    val_len = int(dataset_len * val_ratio)
    test_len = dataset_len - train_len - val_len

    train_dataset_raw, val_dataset_raw, test_dataset_raw = random_split(full_dataset, [train_len, val_len, test_len])

    train_route_files = [route_files[i] for i in train_dataset_raw.indices]
    val_route_files = [route_files[i] for i in val_dataset_raw.indices]
    test_route_files = [route_files[i] for i in test_dataset_raw.indices]

    train_dataset = PathDataset(train_route_files, node_id_to_idx, shuffle_waypoints=True)
    val_dataset = PathDataset(val_route_files, node_id_to_idx, shuffle_waypoints=False)
    test_dataset = PathDataset(test_route_files, node_id_to_idx, shuffle_waypoints=False)

    return train_dataset, val_dataset, test_dataset

In [5]:
class GTN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers,
                 dropout, beta=True, heads=1):
        super(GTN, self).__init__()

        self.num_layers = num_layers

        # Initialize transformer convolution layers with edge attributes
        conv_layers = [TransformerConv(input_dim, hidden_dim // heads, heads=heads, edge_dim=1, beta=beta)]
        conv_layers += [TransformerConv(hidden_dim, hidden_dim // heads, heads=heads, edge_dim=1, beta=beta) for _ in range(num_layers - 2)]
        conv_layers.append(TransformerConv(hidden_dim, output_dim, heads=heads, edge_dim=1, beta=beta, concat=True))
        self.convs = torch.nn.ModuleList(conv_layers)

        # Initialize LayerNorm layers for normalization
        norm_layers = [torch.nn.LayerNorm(hidden_dim) for _ in range(num_layers - 1)]
        self.norms = torch.nn.ModuleList(norm_layers)

        self.dropout = dropout

    def reset_parameters(self):
        """Resets parameters for the convolutional and normalization layers."""
        for conv in self.convs:
            conv.reset_parameters()
        for norm in self.norms:
            norm.reset_parameters()

    def forward(self, x, edge_index, edge_attr):
        """
        Forward pass with edge attributes.
        - x: Node features
        - edge_index: Edge indices
        - edge_attr: Edge attributes
        """
        for i in range(self.num_layers - 1):
            x = self.convs[i](x, edge_index, edge_attr)  # Include edge_attr
            x = self.norms[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)

        # Last layer, average multi-head output.
        x = self.convs[-1](x, edge_index, edge_attr)  # Include edge_attr

        return x

In [6]:
class NodeTransformer(nn.Module):
    def __init__(self, embed_dim, num_heads, num_layers, max_seq_len=48, ff_dim=2048, dropout=0.1):
        super(NodeTransformer, self).__init__()
        
        self.max_seq_len = max_seq_len
        self.embed_dim = embed_dim
        
        # Learnable special embeddings for fixed start and end node
        self.start_node_embed_tag = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.end_node_embed_tag = nn.Parameter(torch.randn(1, 1, embed_dim))
        
        # Transformer encoder layers
        self.encoder_layers = nn.ModuleList([
            nn.TransformerEncoderLayer(
                d_model=embed_dim, 
                nhead=num_heads, 
                dim_feedforward=ff_dim, 
                dropout=dropout, 
                activation='gelu'
            ) 
            for _ in range(num_layers)
        ])
        
        # LayerNorm to stabilize the output
        self.norm = nn.LayerNorm(embed_dim)

        self.head = nn.Linear(embed_dim, 1)

    def forward(self, waypoint_node_embeds, start_node_embed, end_node_embed):
        """
        Args:
            waypoint_node_embeds: Tensor of shape (batch_size, seq_len, embed_dim), where seq_len <= max_seq_len.
            start_node_embed: Tensor of shape (batch_size, 1, embed_dim) representing the first fixed node embedding.
            end_node_embed: Tensor of shape (batch_size, 1, embed_dim) representing the second fixed node embedding.

        Returns:
            Tensor of shape (batch_size, seq_len + 2, embed_dim).
        """
        batch_size, seq_len, embed_dim = waypoint_node_embeds.shape
        
        assert seq_len <= self.max_seq_len, f"Sequence length should be <= {self.max_seq_len}"
        assert embed_dim == self.embed_dim, f"Embedding dimension mismatch: {embed_dim} != {self.embed_dim}"

        # Add learnable tags to fixed nodes
        start_node_embed = start_node_embed + self.start_node_embed_tag  # Shape: (batch_size, 1, embed_dim)
        end_node_embed = end_node_embed + self.end_node_embed_tag  # Shape: (batch_size, 1, embed_dim)

        # Concatenate fixed nodes with the variable-length sequence
        fixed_nodes = torch.cat([start_node_embed, end_node_embed], dim=1)  # Shape: (batch_size, 2, embed_dim)
        full_sequence = torch.cat([fixed_nodes, waypoint_node_embeds], dim=1)  # Shape: (batch_size, seq_len+2, embed_dim)
        
        # Pass through the Transformer encoder layers
        x = full_sequence
        for layer in self.encoder_layers:
            x = layer(x)
        
        # Apply LayerNorm
        x = self.norm(x)
        x = self.head(x)
        
        return x

In [75]:
# Load data and graph
batch_size = 1
graph_path = "data/stanford.pbf"
route_dir = "dataprocessing/out"
graph, node_id_to_idx = construct_graph(graph_path)
train_dataset, val_dataset, test_dataset = load_path_data(route_dir=route_dir, node_id_to_idx=node_id_to_idx)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
print(f"train_dataset length: {len(train_dataset)}")
print(f"val_dataset length: {len(val_dataset)}")
print(f"test_dataset length: {len(test_dataset)}")

train_dataset length: 785
val_dataset length: 98
test_dataset length: 99


In [76]:
class GTTP(nn.Module):
    def __init__(self):
        super(GTTP, self).__init__()

        self.gtn = GTN(input_dim=2, hidden_dim=10, output_dim=10, num_layers=2, dropout=0.1, beta=True, heads=1)
        self.node_transformer_model = NodeTransformer(embed_dim=embed_dim, num_heads=1, num_layers=4)
    
    def forward(self, x, edge_index, edge_attr, start_idx, end_idx, waypoint_node_indices):
        node_embeddings = self.gtn(x, edge_index, edge_attr)
        # node_embeddings = self.gtn()
        start_node_embed = node_embeddings[start_idx].unsqueeze(1)
        end_node_embed = node_embeddings[end_idx].unsqueeze(1)
        waypoint_node_embeds = node_embeddings[waypoint_node_indices]
        pred = self.node_transformer_model(waypoint_node_embeds, start_node_embed, end_node_embed)
        return pred

In [77]:
model = GTTP()

In [85]:
import torch
import torch.nn as nn

class StabilizedKendallTauLoss(nn.Module):
    def __init__(self):
        super(StabilizedKendallTauLoss, self).__init__()
    
    def forward(self, y_pred, y_true):
        """
        Stabilized Kendall Tau Loss (differentiable approximation)
        """
        y_pred = y_pred.float()
        y_true = y_true.float()
        
        # Normalize predictions
        y_pred = (y_pred - y_pred.mean(dim=1, keepdim=True)) / (y_pred.std(dim=1, keepdim=True) + 1e-8)
        
        # Pairwise differences
        diff_pred = y_pred.unsqueeze(2) - y_pred.unsqueeze(1)
        diff_true = y_true.unsqueeze(2) - y_true.unsqueeze(1)
        
        # Clamping to prevent overflow
        diff_pred = torch.clamp(diff_pred, -15, 15)
        
        # Pairwise product
        pairwise_product = diff_true * diff_pred
        
        # Stabilized sigmoid loss
        loss = torch.log1p(torch.exp(-pairwise_product))
        
        # Masking diagonal
        batch_size, num_items, _ = loss.shape
        mask = ~torch.eye(num_items, dtype=torch.bool, device=loss.device)
        mask = mask.unsqueeze(0).expand(batch_size, -1, -1)
        
        loss = loss[mask].view(batch_size, -1)
        
        return loss.mean()


y_pred = torch.tensor([[100000.0, -100000.0, 500.0], [-200000.0, 4000.0, 3000.0]])  # Extreme predictions
y_true = torch.tensor([[3.0, 1.0, 2.0], [2.0, 4.0, 1.0]])  # True scores

loss_fn = StabilizedKendallTauLoss()
loss = loss_fn(y_pred, y_true)
print(f"Stabilized Kendall Tau Loss: {loss.item():.4f}")


Stabilized Kendall Tau Loss: 0.5412


In [ ]:
y_pred = torch.tensor([[-0.4972, 1, 1]])
y_true = torch.tensor([[ 4465.,  1535., 15142.]])

loss = loss_fn(y_pred, y_true)
print(f"Stabilized Kendall Tau Loss: {loss.item():.4f}")

Stabilized Kendall Tau Loss: 0.0000


In [79]:
import torch
import torch.nn as nn

class KendallTauLoss(nn.Module):
    def __init__(self):
        super(KendallTauLoss, self).__init__()
    
    def forward(self, y_pred, y_true):
        """
        Kendall Tau Loss (differentiable approximation)
        
        Parameters:
        - y_pred: Predicted scores (torch.Tensor of shape [batch_size, num_items])
        - y_true: True scores or rankings (torch.Tensor of shape [batch_size, num_items])
        
        Returns:
        - loss: Computed Kendall Tau loss (scalar)
        """
        # Ensure the inputs are float tensors
        y_pred = y_pred.float()
        y_true = y_true.float()
        
        # Create pairwise differences for predictions and ground truth
        diff_pred = y_pred.unsqueeze(2) - y_pred.unsqueeze(1)  # Shape: [batch_size, num_items, num_items]
        diff_true = y_true.unsqueeze(2) - y_true.unsqueeze(1)  # Shape: [batch_size, num_items, num_items]
        
        # Compute pairwise agreements: (y_i - y_j) * (s_i - s_j)
        pairwise_product = diff_true * diff_pred  # Shape: [batch_size, num_items, num_items]
        
        # Compute pairwise loss using logistic sigmoid approximation
        loss = torch.log(1 + torch.exp(-pairwise_product))
        
        # Exclude diagonal elements (self-comparisons)
        batch_size, num_items, _ = loss.shape
        mask = ~torch.eye(num_items, dtype=torch.bool, device=loss.device)  # Shape: [num_items, num_items]
        mask = mask.unsqueeze(0).expand(batch_size, -1, -1)

        loss = loss[mask].view(batch_size, -1)
        
        # Return the mean loss
        return loss.mean()

In [81]:
import torch.nn as nn
import torch.optim as optim

# Define optimizer
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Define the number of epochs
num_epochs = 1

# Move model to appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define a proper loss function
loss_fn = StabilizedKendallTauLoss()

# Training loop
for epoch in range(num_epochs):
    model.train()  # Set model to training mode
    total_loss = 0

    for batch in train_loader:
        start_idx, waypoints_shuffled, waypoints_correct, end_idx = [x.to(device) for x in batch]

        # Forward pass
        predicted_ordering = model(
            graph.x, graph.edge_index, graph.edge_attr, start_idx, end_idx, waypoints_shuffled
        )

        # Process the predictions
        predicted_ordering = predicted_ordering[:, 2:].squeeze(2)

        print(predicted_ordering)
        print(waypoints_correct.float())
        break

        # Compute loss
        loss = loss_fn(predicted_ordering, waypoints_correct.float())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate loss for tracking
        total_loss += loss.item()

    # Print epoch loss
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")


tensor([[-0.4972, -0.9163, -0.4660, -0.7453, -0.7843, -0.9008, -0.7679, -1.4530]],
       grad_fn=<SqueezeBackward1>)
tensor([[ 4465.,  1535., 15142., 16012., 17906., 18829., 19166., 21121.]])
Epoch [1/1], Loss: 0.0000


In [12]:
import torch.optim as optim

# Define optimizer
optimizer = optim.Adam(node_transformer_model.parameters(), lr=1e-3)

# Define the number of epochs
num_epochs = 10

# Move model to appropriate device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
node_transformer_model.to(device)
loss_fn = nn.MSELoss()

# Training loop
for epoch in range(num_epochs):
    node_transformer_model.train()  # Set model to training mode
    total_loss = 0

    for batch in train_loader:
        start_idx, waypoints_shuffled, waypoints_correct, end_idx = [x.to(device) for x in batch]

        # Forward pass
        predicted_ordering = node_transformer_model(
            node_embeddings[waypoints_shuffled], 
            node_embeddings[start_idx].unsqueeze(1), 
            node_embeddings[end_idx].unsqueeze(1)
        )

        # Process the predictions
        predicted_ordering = predicted_ordering[:, 2:].squeeze(2)

        # Compute loss
        # loss = kendall_tau_loss(predicted_ordering, waypoints_correct)

        loss = loss_fn(predicted_ordering, waypoints_correct.float())

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate loss for tracking
        total_loss += loss.item()

    # Print epoch loss
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [26]:
node_embeddings[waypoints_shuffled].shape
node_embeddings[start_idx].shape

torch.Size([32, 10])

In [38]:
start_node_idx = 13
end_node_idx = 14
waypoint_node_indices = [10, 20, 30]

start_node_embed = node_embeddings[start_node_idx].unsqueeze(0).unsqueeze(0)
end_node_embed = node_embeddings[end_node_idx].unsqueeze(0).unsqueeze(0)
waypoint_node_embeds = node_embeddings[waypoint_node_indices].unsqueeze(0)

In [39]:
output = node_transformer_model(waypoint_node_embeds, start_node_embed, end_node_embed)
output.shape

torch.Size([1, 5, 10])